## Stockholms stad 
* Issue [#10](https://github.com/salgo60/Dogpark_Sweden/issues/10)
Hundrastgard_Yta.gpkg

In [1]:
import geopandas as gpd

# Läs in geopackage-filen
gdf = gpd.read_file("Hundrastgard_Yta.gpkg")

# Visa grundläggande information
print(gdf.head())
print(gdf.crs)
print(gdf.columns)


   Anläggningsår      Area   CHANGE_DATE           CID   CREATE_DATE  \
0            NaN  4828.314  2.020051e+11  1.738443e+15  2.020051e+11   
1            NaN   982.746  2.020051e+11  1.738448e+15  2.020051e+11   
2            NaN   950.420  2.020051e+11  1.738446e+15  2.020051e+11   
3         2011.0   997.487  2.020051e+11  1.738443e+15  2.020051e+11   
4         2018.0   200.337  2.022032e+11  1.738442e+15  2.020051e+11   

   FEATURE_TYPE_OBJECT_ID  FEATURE_TYPE_VERSION_ID Huvudman  \
0                17314453                        1      SdF   
1                17314453                        1      SdF   
2                17314453                        1      SdF   
3                17314453                        1      SdF   
4                17314453                        1      SdF   

                                       ID MAIN_ATTRIBUTE_DESCRIPTION  \
0                                    None                        SdF   
1                                 Ekhagen   

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: GPKG: unrecognized user_version=0x00000000 (0) on 'Hundrastgard_Yta.gpkg'
  return ogr_read(


In [11]:
def convert_misread_date(x):
    try:
        if pd.isna(x):
            return None
        # gör till int, ta bara siffrorna
        s = str(int(float(x)))
        # om 8 tecken → YYYYMMDD
        if len(s) == 8:
            return pd.to_datetime(s, format="%Y%m%d", errors="coerce")
        return None
    except Exception:
        return None

for col in [c for c in gdf.columns if "DATE" in c.upper()]:
    gdf[col + "_fmt"] = gdf[col].apply(convert_misread_date)


In [9]:
import geopandas as gpd
import pandas as pd

# Läs in
gdf = gpd.read_file("Hundrastgard_Yta.gpkg")

# Beräkna centroid i meter-CRS (SWEREF99 TM) och konvertera till WGS84
gdf_proj = gdf.to_crs(epsg=3006)
gdf["geometry_centroid"] = gdf_proj.geometry.centroid.to_crs(epsg=4326)

# Lägg till lat/lon
gdf["lon"] = gdf["geometry_centroid"].x
gdf["lat"] = gdf["geometry_centroid"].y

# Lista möjliga datumkolumner
date_cols = [c for c in gdf.columns if "DATE" in c.upper() or "DATUM" in c.upper()]
print("🕒 Datumkolumner:", date_cols)

# Funktion för att försöka tolka stora siffror som tidsstämplar
def convert_date(x):
    try:
        # Om NaN
        if pd.isna(x):
            return None
        # Om redan datetime
        if isinstance(x, pd.Timestamp):
            return x
        # Om det ser ut som 1.738443e+15 (millisekunder)
        x = float(x)
        if x > 1e12:  # troligen millisekunder
            return pd.to_datetime(x, unit="ms")
        elif x > 1e9:  # sekunder
            return pd.to_datetime(x, unit="s")
        else:
            return None
    except Exception:
        return None

# Konvertera alla identifierade datumfält
for col in date_cols:
    gdf[col + "_fmt"] = gdf[col].apply(convert_date)

# Visa exempel på konverterade datum
print(gdf[[c for c in gdf.columns if "DATE" in c or "fmt" in c]].head())

gdf.drop(columns=["geometry", "geometry_centroid"]).to_csv("hundrastgardar_fix.csv", index=False)
print("✅ Sparade 'hundrastgardar_fix.csv' med riktiga datum och koordinater")


🕒 Datumkolumner: ['CHANGE_DATE', 'CREATE_DATE']
    CHANGE_DATE   CREATE_DATE CHANGE_DATE_fmt CREATE_DATE_fmt
0  2.020051e+11  2.020051e+11            None            None
1  2.020051e+11  2.020051e+11            None            None
2  2.020051e+11  2.020051e+11            None            None
3  2.020051e+11  2.020051e+11            None            None
4  2.022032e+11  2.020051e+11            None            None
✅ Sparade 'hundrastgardar_fix.csv' med riktiga datum och koordinater


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: GPKG: unrecognized user_version=0x00000000 (0) on 'Hundrastgard_Yta.gpkg'
  return ogr_read(


In [13]:
gdf.info()


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   Anläggningsår               30 non-null     float64 
 1   Area                        88 non-null     float64 
 2   CHANGE_DATE                 88 non-null     float64 
 3   CID                         88 non-null     float64 
 4   CREATE_DATE                 88 non-null     float64 
 5   FEATURE_TYPE_OBJECT_ID      88 non-null     int64   
 6   FEATURE_TYPE_VERSION_ID     88 non-null     int16   
 7   Huvudman                    88 non-null     object  
 8   ID                          29 non-null     object  
 9   MAIN_ATTRIBUTE_DESCRIPTION  88 non-null     object  
 10  MAIN_ATTRIBUTE_VALUE        88 non-null     object  
 11  OBJECT_ID                   88 non-null     int64   
 12  VALID_FROM                  88 non-null     float64 
 13  VALID_TO      

In [15]:
gdf.describe

<bound method NDFrame.describe of     Anläggningsår      Area   CHANGE_DATE           CID   CREATE_DATE  \
0             NaN  4828.314  2.020051e+11  1.738443e+15  2.020051e+11   
1             NaN   982.746  2.020051e+11  1.738448e+15  2.020051e+11   
2             NaN   950.420  2.020051e+11  1.738446e+15  2.020051e+11   
3          2011.0   997.487  2.020051e+11  1.738443e+15  2.020051e+11   
4          2018.0   200.337  2.022032e+11  1.738442e+15  2.020051e+11   
..            ...       ...           ...           ...           ...   
83            NaN  2417.832  2.023092e+11  2.084623e+15  2.023092e+11   
84         2021.0  1408.689  2.024011e+11  2.127269e+15  2.024011e+11   
85            NaN   201.880  2.025082e+11  2.128834e+15  2.024012e+11   
86         2019.0   462.869  2.024101e+11  1.738447e+15  2.020051e+11   
87         2024.0   435.672  2.024121e+11  2.255248e+15  2.024121e+11   

    FEATURE_TYPE_OBJECT_ID  FEATURE_TYPE_VERSION_ID Huvudman  \
0                 1731445

In [19]:
from shapely.geometry import base

for col in gdf.columns:
    has_geom = gdf[col].apply(lambda x: isinstance(x, base.BaseGeometry)).any()
    if has_geom:
        print(f"🧱 Kolumn '{col}' innehåller geometrier")


🧱 Kolumn 'geometry' innehåller geometrier


In [20]:
from shapely.geometry import base

# Ta bort alla geometrier utom den huvudsakliga
gdf_clean = gdf.copy()

# ta bort "geometry_centroid" och andra geometrikolumner
geom_cols = [c for c in gdf_clean.columns 
             if c != "geometry" and gdf_clean[c].apply(lambda x: isinstance(x, base.BaseGeometry)).any()]
if geom_cols:
    print("🧹 Tar bort extra geometri-kolumner:", geom_cols)
    gdf_clean = gdf_clean.drop(columns=geom_cols, errors="ignore")

# Skapa Folium-lager utan att försöka serialisera själva geometry-kolumnen
stad_layer = folium.FeatureGroup(name="🟠 Hundrastgårdar (Stockholms stad)")

folium.GeoJson(
    data=gdf_clean.__geo_interface__,   # endast den officiella geometrin används
    name="🟠 Hundrastgårdar (Stockholms stad)",
    style_function=lambda feature: {
        "fillColor": "#ff6600",
        "color": "#cc5500",
        "weight": 2,
        "fillOpacity": 0.3,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=[c for c in gdf_clean.columns if c != "geometry"],
        aliases=[f"{c}:" for c in gdf_clean.columns if c != "geometry"],
        sticky=False,
        localize=True,
    )
).add_to(stad_layer)

stad_layer.add_to(m)


In [22]:
m


In [21]:
from IPython.display import display
display(gdf.head(200))

,Anläggningsår,Area,CHANGE_DATE,CID,CREATE_DATE,FEATURE_TYPE_OBJECT_ID,FEATURE_TYPE_VERSION_ID,Huvudman,ID,MAIN_ATTRIBUTE_DESCRIPTION,...,VALID_FROM,VALID_TO,VERSION_ID,fid,FEATURE_TYPE_NAME,geometry,lon,lat,CHANGE_DATE_fmt,CREATE_DATE_fmt
0,NaN,4828.314,2.020051e+11,1.738443e+15,2.020051e+11,17314453,1,SdF,None,SdF,...,2.020051e+11,None,1,1,Hundrastgård,"POLYGON ((157107.699 6572851.359, 157214.251 6...",18.125681,59.271015,None,None
1,NaN,982.746,2.020051e+11,1.738448e+15,2.020051e+11,17314453,1,SdF,Ekhagen,SdF,...,2.020051e+11,None,1,2,Hundrastgård,"POLYGON ((153213.019 6583894.338, 153245.874 6...",18.056876,59.370128,None,None
2,NaN,950.420,2.020051e+11,1.738446e+15,2.020051e+11,17314453,1,SdF,Hundrastgård Sjösavägen,SdF,...,2.020051e+11,None,1,3,Hundrastgård,"POLYGON ((152576.332 6572381.232, 152597.096 6...",18.045433,59.266931,None,None
3,2011.0,997.487,2.020051e+11,1.738443e+15,2.020051e+11,17314453,1,SdF,None,SdF,...,2.020051e+11,None,1,4,Hundrastgård,"POLYGON ((154670.653 6571689.547, 154663.07 65...",18.081647,59.260265,None,None
4,2018.0,200.337,2.022032e+11,1.738442e+15,2.020051e+11,17314453,1,SdF,"HRG 2 Blackeberg, Blackebergsbacken 60",SdF,...,2.020051e+11,None,1,5,Hundrastgård,"POLYGON ((143032.16 6580985.794, 143027.776 65...",17.877582,59.343793,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,NaN,2417.832,2.023092e+11,2.084623e+15,2.023092e+11,17314453,1,SdF,None,SdF,...,2.023092e+11,None,1,84,Hundrastgård,"POLYGON ((146396.799 6587513.209, 146395.786 6...",17.936994,59.402349,None,None
84,2021.0,1408.689,2.024011e+11,2.127269e+15,2.024011e+11,17314453,1,SdF,None,SdF,...,2.024011e+11,None,1,85,Hundrastgård,"POLYGON ((156169.456 6571715.112, 156175.872 6...",18.108547,59.260518,None,None
85,NaN,201.880,2.025082e+11,2.128834e+15,2.024012e+11,17314453,1,SdF,None,SdF,...,2.024012e+11,None,1,86,Hundrastgård,"POLYGON ((152373.467 6581523.715, 152374.431 6...",18.041889,59.348829,None,None
86,2019.0,462.869,2.024101e+11,1.738447e+15,2.020051e+11,17314453,1,SdF,"HRG 5 Traneberg, Tranebergsv 8",SdF,...,2.020051e+11,None,1,87,Hundrastgård,"POLYGON ((149551.348 6579922.946, 149540.914 6...",17.991851,59.334466,None,None
